<a href="https://colab.research.google.com/github/sanAkel/ocean-hurricane/blob/main/aviso_cmems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To download (L4) SSH data from [AVISO/CMEMS.](https://data.marine.copernicus.eu/products?facets=mainVariables%7ESea+surface+height--areas%7EGlobal+Ocean--tempResolutions%7EDaily--sources%7ESatellite+observations)

In [ ]:
# User inputs, should be consistent with those in:
# https://github.com/sanAkel/ocean-hurricane/blob/main/get_track.ipynb

# Basin, year and category
myBasin = 'north_atlantic'
year = 2021
cat_threshold = 4

day_buffer = 2 # extract CMEMS data hurricane start day-day_buffer --> end day+day_buffer

In [ ]:
if year < 2022:
  dsetID = "c3s_obs-sl_glo_phy-ssh_my_twosat-l4-duacs-0.25deg_P1D"
  vNames = ["adt", "sla", "err_sla", "ugos", "vgos"]
else:
  dsetID = "cmems_obs-sl_glo_phy-ssh_nrt_allsat-l4-duacs-0.25deg_P1D"
  vNames = ["adt", "sla", "err_sla", "ugos", "vgos"]

## Install API

In [ ]:
!pip install copernicusmarine

In [ ]:
!pip install cartopy

In [ ]:
import copernicusmarine

import pandas as pd
import numpy as np
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import glob as glob

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

## Set up environment for the [Copernicus Marine Toolbox.](https://help.marine.copernicus.eu/en/articles/7949409-copernicus-marine-toolbox-introduction)

In [ ]:
print(copernicusmarine.__version__)
copernicusmarine.login(username="sakella", password="HbFPyP9M")

In [ ]:
# Copernicus marine dataset
# 1992- 2023 (reprocessed): https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_CLIMATE_L4_MY_008_057/description
# Choose: c3s_obs-sl_glo_phy-ssh_my_twosat-l4-duacs-0.25deg_P1D,
# not `c3s_obs-sl_glo_phy-ssh_myint_twosat-l4-duacs-0.25deg_P1D` which has last 1 month.
#
# 2021- present: https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_L4_NRT_008_046/description
# ----
# Which ocean dataset to read?
# Hourly mean surface (2d) fields: cmems_mod_glo_phy_anfc_0.083deg_PT1H-m
# Instantaneous (inst) 6-hourly 3d potential temperature: cmems_mod_glo_phy-thetao_anfc_0.083deg_PT6H-i
# inst 6hr 3d salinity: cmems_mod_glo_phy-so_anfc_0.083deg_PT6H-i
# inst 6hr 3d currents: cmems_mod_glo_phy-cur_anfc_0.083deg_PT6H-i
# ----

# Set parameters
data_request = {
    "dataset_id" : dsetID,
    "longitude" : [-180, 180],
    "latitude" : [-80, 90],
    "variables" : vNames
}

In [ ]:
cms_data =copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    variables = data_request["variables"]
)

## Read the processed/downloaded hurricane data. It was generated using [this notebook.](https://github.com/sanAkel/ocean-hurricane/blob/main/get_track.ipynb)

In [ ]:
# Mount drive - to save files once done
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Read downloaded track data, subset and save CMEMS data for the hurricane dates
input_data_path = '/content/drive/MyDrive/datasets/hurr/{}/'.format(year)
input_fName = input_data_path + 'hurdat2_{}_{}.csv'.format(myBasin, year)
print("Reading {} summary data from:\n{}".format(year, input_fName))

season_data=pd.read_csv(input_fName)
major_hurr_names = season_data['name'][season_data['category'] >=cat_threshold]
major_hurr_ids = season_data['id'][season_data['category'] >=cat_threshold]

print("\n\nStorms that had a category >= {} are following:\n".format(cat_threshold))
#for hurr in major_hurr_names:
#    print(hurr)

# Read storm data
for idx, hurr_id in enumerate(major_hurr_ids):
    print("{}, ID: [{}]".format(major_hurr_names.iloc[idx], hurr_id))
    input_hurr_file = str(year) + "_" + myBasin + "_" + major_hurr_names.iloc[idx] + '.nc'
    print("Processed file name:\t{}".format(input_data_path + input_hurr_file))
    track_ds = xr.open_dataset(input_data_path + input_hurr_file)
    print("Start and end days:\n{}--\t{}.\n".format(track_ds.time[0].values, track_ds.time[-1].values))

    ts, te = [track_ds.time[0].values- np.timedelta64(day_buffer, 'D'), track_ds.time[-1].values + np.timedelta64(day_buffer, 'D')]
    hurr_subset=cms_data.sel(time=slice(ts, te))

    output_hurr_file = str(year) + "_" + myBasin + "_" + major_hurr_names.iloc[idx] + '_AVISO_L4.nc'
    print("Output file name:\t{}\n\n".format(input_data_path + output_hurr_file))
    hurr_subset.to_netcdf(input_data_path + output_hurr_file)